In [1]:
from llama_index.readers.file import  PandasCSVReader
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from llama_index.core.agent.workflow import FunctionAgent
from dotenv import load_dotenv
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import Settings
import os
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.tools import QueryEngineTool
from llama_index.core.workflow import Context
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.groq import Groq

c:\Users\mateo\Desktop\Semestre 8\Banco-w\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [44]:
load_dotenv()
openai_token = os.getenv("OPENAI_API_KEY")
gemini_token = os.getenv("GOOGLE_API_KEY")
groq_token = os.getenv("GROQ_API_KEY")
llm = Groq(model="llama3-70b-8192", api_key=groq_token)
embed_model = GoogleGenAIEmbedding(model="gemini-embedding-exp-03-07", token=gemini_token)

In [46]:
Settings.llm= llm
Settings.embed_model = embed_model

try:

    storage_context = StorageContext.from_defaults(
        persist_dir="../storage/ciuu"
    )
    ciuu_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [47]:
parser = PandasCSVReader(pandas_config={"encoding": "latin1"})
file_extractor = {".csv": parser} 
if not index_loaded:
    ciuu_docs = SimpleDirectoryReader(
        "../data", file_extractor=file_extractor
    ).load_data()

    ciuu_index = VectorStoreIndex.from_documents(ciuu_docs)

    ciuu_index.storage_context.persist(persist_dir="../storage/ciuu")


In [48]:
ciuu_engine = ciuu_index.as_query_engine(similarity_top_k=3, structured_output=True)

In [49]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=ciuu_engine,
        name="ciuu",
        description="use this tool to get information from the ciuu database",
    )
]

In [50]:
ciuu_agent = FunctionAgent(
    name = "ciuu",
    description="use this agent to get information from the ciuu database",
    tools=query_engine_tools,
    llm=llm,
    system_prompt="""
                    Eres un asistente experto en clasificación económica que ayuda a las personas a identificar los códigos CIIU (Clasificación Industrial Internacional Uniforme) que mejor se ajustan a la descripción de su actividad económica.

                    Cuando el usuario proporcione una descripción de su trabajo, analiza cuidadosamente el texto para entender la actividad principal. Luego, busca y selecciona los códigos CIIU más relevantes según su similitud semántica y conceptual.

                    Tu respuesta debe ser un objeto JSON válido con una lista llamada "resultados", en la que cada elemento contiene dos claves: "codigo" y "descripcion".

                    Devuelve siempre 3 coincidencias relevantes, ordenadas por similitud. No incluyas texto adicional, explicaciones ni comentarios fuera del JSON. Devuelve solo el JSON sin formatos como  ```json ``` ni cosas asi.
                    Formato requerido:
                    {
                    "opcion_1": {
                    "codigo": "0112",
                    "descripcion": "Cultivo de arroz"
                    },
                    "opcion_2": {
                    "codigo": "0113",
                    "descripcion": "Cultivo de hortalizas, raíces y tubérculos"
                    },
                    "opcion_3": {
                    "codigo": "0111",
                    "descripcion": "Cultivo de cereales (excepto arroz), legumbres y semillas oleaginosas"
                    }
                }

                    Si la descripción es ambigua o insuficiente, devuelve exactamente esto:
                    {
                    "resultados": []
                    }
                    """
)



In [51]:
handler = ciuu_agent.run("Yo vendo pollo, carne y queso en mi negocio")

response = await handler

In [52]:
print(handler)

{
"opcion_1": {
"codigo": "4723",
"descripcion": "Comercio al por menor de carnes (incluye aves de corral), productos cárnicos, pescados y productos de mar, en establecimientos especializados"
},
"opcion_2": {
"codigo": "4729",
"descripcion": "Comercio al por menor de productos lácteos, huevos, aceites y grasas comestibles, en establecimientos especializados"
},
"opcion_3": {
"codigo": "5211",
"descripcion": "Comercio al por mayor de carnes, productos cárnicos, pescados y productos de mar"
}
}
